## Секреты Темнолесья

### Цели и задачи проекта

<font color='#777778'>Основная цель проекта: привлечение новой аудитории с помощью исследования развития игровой индустрии с 2000 по 2013 годы
Задачи:
1. Обзор игровх платформ
2. Изучение объёма продаж игр разных жанров
3. Региональные предпочтения игроков
4. Акцент на игры жанра RPG (компьютерные ролевые игры)</font>

### Описание данных

<font color='#777778'>В проекте будут использованы данные датасета `/datasets/new_games.csv` с таким описанием:

- `Name` — название игры.
- `Platform` — название платформы.
- `Year of Release` — год выпуска игры.
- `Genre` — жанр игры.
- `NA sales` — продажи в Северной Америке (в миллионах проданных копий).
- `EU sales` — продажи в Европе (в миллионах проданных копий).
- `JP sales` — продажи в Японии (в миллионах проданных копий).
- `Other sales` — продажи в других странах (в миллионах проданных копий).
- `Critic Score` — оценка критиков (от 0 до 100).
- `User Score` — оценка пользователей (от 0 до 10).
- `Rating` — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.
</font>

### Содержимое проекта
---

    
[1. Загрузка данных и знакомство с ними](#1.-Загрузка-данных-и-знакомство-с-ними)
    
[2. Проверка ошибок в данных и их предобработка](#2.-Проверка-ошибок-в-данных-и-их-предобработка)

[2.1. Названия или метки столбцов датафрейма](#2.1.-Названия-или-метки-столбцов-датафрейма)

[2.2. Типы данных](#2.2.-Типы-данных)

[2.3. Наличие пропусков в данных](#2.3.-Наличие-пропусков-в-данных)

[2.4. Явные и неявные дубликаты в данных](#2.4.-Явные-и-неявные-дубликаты-в-данных)
    
[3. Фильтрация данных](#3.-Фильтрация-данных)
    
[4. Категоризация данных](#4.-Категоризация-данных)
    
[5. Итоговый вывод](#5.-Итоговый-вывод)

## 1. Загрузка данных и знакомство с ними

- Загрузите необходимые библиотеки Python и данные датасета `/datasets/new_games.csv`.


In [1]:
#Импортируем библиотеку pandas
import pandas as pd

In [2]:
#Загружаем данные из датасета /datasets/new_games.csv в датафрейм games
new_game = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

In [3]:
#Выводим информацию о датафрейме
new_game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [4]:
# Выводим первые строки датафрейма на экран
new_game.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


Датасет `datasets/new_games.csv` содержит 11 колонок и 16955 строк, в которых представлена информация об игровых платформах.

Изучим типы данных и их корректность:
- **Числовые значения с плавающей запятой (float64).** Четыре столбца: `Year of Release`, `NA sales`, `Other sales`, `Critic Score`, некорректный тип данных в столбце `Year of Release`, по продажам и оценке выводит данных в таком типе правильный и корректный.
- **Строковые данные (object).** Семь столбцов имеют тип данных `object`:
- `Name`, `Platform`, `Genre`, `Rating` содержат строковую информацию (название игры, платформу и жанр), что логично для текстовых данных.
- `EU sales`, `JP sales` хранят информацию о продажах, что несовсем верно, такие формат данных лучше хранить как тип `float64`.
- `User Score`, по моему мнению, должны быть также представлены в формате `float64`, так как хранят в себе информацию об оценке пользователя и рейтинг.

После анализа типов данных видно, что почти половина столбцов некорректно представлена. Для оптимизации необходимо преобразовать все вышеперечисленные данные в соответствующий тип.

### Пропущенные значения:

Исходя из информации о датасете, мы видим, что пропуски присутствуют в 6 столбцах, а именно: `Name`, `Year of Release`, `Genre`, `Critic Score`, `User Score`, `Rating`. Визуально видно, что самое большое количество пропущенных данных в столбце `Critic Score`

---

## 2.  Проверка ошибок в данных и их предобработка


### 2.1. Названия, или метки, столбцов датафрейма

- Выведите на экран названия всех столбцов датафрейма и проверьте их стиль написания.
- Приведите все столбцы к стилю snake case. Названия должны быть в нижнем регистре, а вместо пробелов — подчёркивания.

In [5]:
new_game.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

In [6]:
columns_snake_case = new_game.columns.str.lower().str.replace(' ','_')
new_game.columns = columns_snake_case
new_game.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

### 2.2. Типы данных

- Если встречаются некорректные типы данных, предположите их причины.
- При необходимости проведите преобразование типов данных. Помните, что столбцы с числовыми данными и пропусками нельзя преобразовать к типу `int64`. Сначала вам понадобится обработать пропуски, а затем преобразовать типы данных.

Как я указала выше, по моему мнению некорректные типы данных в столбцах: `'year_of_release'` - его необходимо заменить на тип int64; `'platform', 'genre', 'rating'` - category; `'jp_sales', 'eu_sales', 'user_score'` - в тип данных float64

In [7]:
# Преобразовываем типы данных к типу float64
new_game[['eu_sales', 'jp_sales','user_score']] = new_game[['eu_sales', 'jp_sales','user_score']].apply(pd.to_numeric, errors='coerce')

In [8]:
# Преобразовываем типы данных к типу category
new_game[['platform', 'genre', 'rating']] = new_game[['platform', 'genre', 'rating']].astype('category')

In [9]:
# Преобразовываем типы данных к типу int64
new_game['year_of_release'] = new_game['year_of_release'].astype('Int64')
new_game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16954 non-null  object  
 1   platform         16956 non-null  category
 2   year_of_release  16681 non-null  Int64   
 3   genre            16954 non-null  category
 4   na_sales         16956 non-null  float64 
 5   eu_sales         16950 non-null  float64 
 6   jp_sales         16952 non-null  float64 
 7   other_sales      16956 non-null  float64 
 8   critic_score     8242 non-null   float64 
 9   user_score       7688 non-null   float64 
 10  rating           10085 non-null  category
dtypes: Int64(1), category(3), float64(6), object(1)
memory usage: 1.1+ MB


- В числовых столбцах могут встретиться строковые значения, например `unknown` или другие. Приводите такие столбцы к числовому типу данных, заменив строковые значения на пропуски.

### 2.3. Наличие пропусков в данных

- Посчитайте количество пропусков в каждом столбце в абсолютных и относительных значениях.


In [10]:
# Абсолютные значения
new_game.isna().sum()

name                  2
platform              0
year_of_release     275
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

In [11]:
# Относительные значения (в процентах)
new_game.isna().mean() * 100

name                0.011795
platform            0.000000
year_of_release     1.621845
genre               0.011795
na_sales            0.000000
eu_sales            0.035386
jp_sales            0.023590
other_sales         0.000000
critic_score       51.391838
user_score         54.659118
rating             40.522529
dtype: float64

Исходя из данных выше в столбцах `name`, `genre`, `eu_sales`, `jp_sales` содержат менее 10 пропусков, исходя из общего количества строк (16956), это составляет менее 0,06 процента, что говорит о том, эти пропуски можно оставить как есть или удалить

В столбце `year_of_release` 275 пропусков, это составляет 1,6 % от общего количетсва данных, можно заполнить, но также можно оставить этот столбец без изменений

В столбце `rating` 6871 пропущенных значений 40%, доля пропусков высокая, но я бы заполнила их значением, например -1

В столбцах `user_score`, `critic_score` пропусков более 50%, то есть больше половины данных, я бы их попробовала заполнить медианным значением

Ответ на вопрос, почему могли появиться пропуски, это могло произойти по причине руного ввода информации, либо при переносе данных возникла ошибка


In [12]:
# Добавляем новое значение в список категорий
if -1 not in new_game['rating'].cat.categories:
    new_game['rating'].cat.add_categories([-1], inplace=True)
else:
    # Выбираем другое значение, например, -999
    new_game['rating'].cat.add_categories([-999], inplace=True)
new_game

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16951,Samurai Warriors: Sanada Maru,PS3,2016,Action,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16952,LMA Manager 2007,X360,2006,Sports,0.00,0.01,0.00,0.00,NaN,NaN,NaN
16953,Haitaka no Psychedelica,PSV,2016,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16954,Spirits & Spells,GBA,2003,Platform,0.01,0.00,0.00,0.00,NaN,NaN,NaN


In [13]:
#Заменяем заглушками в столбцах по году и оценкам на значение в пропусках
new_game[['year_of_release', 'critic_score', 'user_score', 'rating']] = new_game[['year_of_release', 'critic_score', 'user_score', 'rating']].fillna(-1)
new_game

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,-1.0,-1.0,-1
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,-1.0,-1.0,-1
...,...,...,...,...,...,...,...,...,...,...,...
16951,Samurai Warriors: Sanada Maru,PS3,2016,Action,0.00,0.00,0.01,0.00,-1.0,-1.0,-1
16952,LMA Manager 2007,X360,2006,Sports,0.00,0.01,0.00,0.00,-1.0,-1.0,-1
16953,Haitaka no Psychedelica,PSV,2016,Adventure,0.00,0.00,0.01,0.00,-1.0,-1.0,-1
16954,Spirits & Spells,GBA,2003,Platform,0.01,0.00,0.00,0.00,-1.0,-1.0,-1


### 2.4. Явные и неявные дубликаты в данных

- Изучите уникальные значения в категориальных данных, например с названиями жанра игры, платформы, рейтинга и года выпуска. Проверьте, встречаются ли среди данных неявные дубликаты, связанные с опечатками или разным способом написания.
- При необходимости проведите нормализацию данных с текстовыми значениями. Названия или жанры игр можно привести к нижнему регистру, а названия рейтинга — к верхнему.

In [14]:
print('Количество дублирующихся строк:', new_game.duplicated().sum())

new_game.nunique()

Количество дублирующихся строк: 182


name               11559
platform              31
year_of_release       38
genre                 24
na_sales             402
eu_sales             307
jp_sales             244
other_sales          155
critic_score          83
user_score            96
rating                 9
dtype: int64

In [15]:
# Для каждого категориального столбца
for column in ['genre', 'platform', 'rating', 'year_of_release']:
    print(f"\nУникальные значения в {column}:")
    print(new_game[column].unique())
    print(f"Количество уникальных значений: {new_game[column].nunique()}")


Уникальные значения в genre:
['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', ..., 'PLATFORM', 'ADVENTURE', 'SIMULATION', 'PUZZLE', 'STRATEGY']
Length: 25
Categories (24, object): ['Sports', 'Platform', 'Racing', 'Role-Playing', ..., 'ADVENTURE', 'SIMULATION', 'PUZZLE', 'STRATEGY']
Количество уникальных значений: 24

Уникальные значения в platform:
['Wii', 'NES', 'GB', 'DS', 'X360', ..., 'NG', 'TG16', '3DO', 'GG', 'PCFX']
Length: 31
Categories (31, object): ['Wii', 'NES', 'GB', 'DS', ..., 'TG16', '3DO', 'GG', 'PCFX']
Количество уникальных значений: 31

Уникальные значения в rating:
['E', -1, 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP']
Categories (9, object): ['E', -1, 'M', 'T', ..., 'K-A', 'AO', 'EC', 'RP']
Количество уникальных значений: 9

Уникальные значения в year_of_release:
<IntegerArray>
[2006, 1985, 2008, 2009, 1996, 1989, 1984, 2005, 1999, 2007, 2010, 2013, 2004,
 1990, 1988, 2002, 2001, 2011, 1998, 2015, 2012, 2014, 1992, 1997, 1993, 1994,
 1982, 2016, 2003, 1986, 2

In [16]:
# Поиск дубликатов для жанра
genre_counts = new_game['genre'].value_counts().sort_index()
print("Возможные дубликаты в жанрах:")
print(genre_counts[genre_counts > 1])

Возможные дубликаты в жанрах:
ACTION            13
ADVENTURE          4
Action          3405
Adventure       1319
FIGHTING           6
Fighting         856
MISC               3
Misc            1769
PLATFORM           3
PUZZLE             2
Platform         901
Puzzle           588
RACING             6
ROLE-PLAYING       6
Racing          1267
Role-Playing    1510
SHOOTER            5
SIMULATION         2
SPORTS             8
Shooter         1341
Simulation       882
Sports          2367
Strategy         690
Name: genre, dtype: int64


In [17]:
# Поиск дубликатов для платформы
platform_counts = new_game['platform'].value_counts().sort_index()
print("Возможные дубликаты в платформе:")
print(platform_counts[platform_counts > 1])

Возможные дубликаты в платформе:
2600     135
3DO        3
3DS      530
DC        52
DS      2177
GB        98
GBA      837
GC       563
GEN       29
N64      323
NES      100
NG        12
PC       990
PS      1215
PS2     2189
PS3     1355
PS4      395
PSP     1229
PSV      435
SAT      174
SCD        6
SNES     241
TG16       2
WS         6
Wii     1340
WiiU     147
X360    1281
XB       839
XOne     251
Name: platform, dtype: int64


In [18]:
# Поиск дубликатов для рейтинга
rating_counts = new_game['rating'].value_counts().sort_index()
print("Возможные дубликаты в рейтинге:")
print(rating_counts[rating_counts > 1])

Возможные дубликаты в рейтинге:
E       4037
E10+    1441
EC         8
K-A        3
M       1587
RP         3
T       3005
-1      6871
Name: rating, dtype: int64


Нам необходимо в рейтинге заменить значения 'К-А' на 'Е', так как согласно официальному сайту в период 1994-1998 вместо 'Е' было 'К-А', поэтому нам также необходимо привести все данные к одному рейтингу

In [19]:
#Заменяем 'К-А' на 'Е'
new_game['rating'] = new_game['rating'].str.replace('K-A', 'Е')
new_game

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,-1.0,-1.0,NaN
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,-1.0,-1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16951,Samurai Warriors: Sanada Maru,PS3,2016,Action,0.00,0.00,0.01,0.00,-1.0,-1.0,NaN
16952,LMA Manager 2007,X360,2006,Sports,0.00,0.01,0.00,0.00,-1.0,-1.0,NaN
16953,Haitaka no Psychedelica,PSV,2016,Adventure,0.00,0.00,0.01,0.00,-1.0,-1.0,NaN
16954,Spirits & Spells,GBA,2003,Platform,0.01,0.00,0.00,0.00,-1.0,-1.0,NaN


In [20]:
# Поиск дубликатов для года
year_of_release_counts = new_game['year_of_release'].value_counts().sort_index()
print("Возможные дубликаты в годах:")
print(year_of_release_counts[year_of_release_counts > 1])

Возможные дубликаты в годах:
-1        275
 1980       9
 1981      46
 1982      37
 1983      18
 1984      14
 1985      14
 1986      22
 1987      17
 1988      15
 1989      17
 1990      16
 1991      42
 1992      43
 1993      62
 1994     121
 1995     220
 1996     267
 1997     293
 1998     384
 1999     341
 2000     357
 2001     491
 2002     839
 2003     789
 2004     771
 2005     950
 2006    1020
 2007    1218
 2008    1445
 2009    1450
 2010    1279
 2011    1149
 2012     670
 2013     552
 2014     584
 2015     612
 2016     507
Name: year_of_release, dtype: Int64


In [21]:
# Приведение к нижнему регистру
new_game['genre'] = new_game['genre'].str.lower()
new_game['genre'] = new_game['genre'].str.lower()
new_game['platform'] = new_game['platform'].str.lower()

# Приведение рейтинга к верхнему регистру
new_game['rating'] = new_game['rating'].str.upper()

In [22]:
# После нормализации
for column in ['genre', 'platform', 'rating', 'year_of_release']:
    print(f"\nУникальные значения в {column} после очистки:")
    print(new_game[column].unique())
    print(f"Количество уникальных значений: {new_game[column].nunique()}")


Уникальные значения в genre после очистки:
['sports' 'platform' 'racing' 'role-playing' 'puzzle' 'misc' 'shooter'
 'simulation' 'action' 'fighting' 'adventure' 'strategy' nan]
Количество уникальных значений: 12

Уникальные значения в platform после очистки:
['wii' 'nes' 'gb' 'ds' 'x360' 'ps3' 'ps2' 'snes' 'gba' 'ps4' '3ds' 'n64'
 'ps' 'xb' 'pc' '2600' 'psp' 'xone' 'wiiu' 'gc' 'gen' 'dc' 'psv' 'sat'
 'scd' 'ws' 'ng' 'tg16' '3do' 'gg' 'pcfx']
Количество уникальных значений: 31

Уникальные значения в rating после очистки:
['E' nan 'M' 'T' 'E10+' 'Е' 'AO' 'EC' 'RP']
Количество уникальных значений: 8

Уникальные значения в year_of_release после очистки:
<IntegerArray>
[2006, 1985, 2008, 2009, 1996, 1989, 1984, 2005, 1999, 2007, 2010, 2013, 2004,
 1990, 1988, 2002, 2001, 2011, 1998, 2015, 2012, 2014, 1992, 1997, 1993, 1994,
 1982, 2016, 2003, 1986, 2000,   -1, 1995, 1991, 1981, 1987, 1980, 1983]
Length: 38, dtype: Int64
Количество уникальных значений: 38


In [23]:
# Проверка количества строк до и после
print("Количество строк до удаления дубликатов:", len(new_game))
new_game_cleaned = new_game.drop_duplicates()
print("Количество строк после удаления дубликатов:", len(new_game_cleaned))

Количество строк до удаления дубликатов: 16956
Количество строк после удаления дубликатов: 16715


- После того как нормализуете данные и устраните неявные дубликаты, проверьте наличие явных дубликатов в данных.

In [24]:
#Подсчёт количества дубликатов
duplicates = new_game.duplicated()
display(duplicates)
num_duplicates = duplicates.sum()
display(f"Количество дубликатов: {num_duplicates}")
duplicate_rows = new_game[duplicates]
display(duplicate_rows)

0        False
1        False
2        False
3        False
4        False
         ...  
16951    False
16952    False
16953    False
16954    False
16955    False
Length: 16956, dtype: bool

'Количество дубликатов: 241'

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
268,Batman: Arkham Asylum,ps3,2009,action,2.24,1.31,0.07,0.61,91.0,8.9,T
368,James Bond 007: Agent Under Fire,ps2,2001,shooter,1.90,1.13,0.10,0.41,72.0,7.9,T
717,God of War: Ascension,ps3,2013,action,1.23,0.63,0.04,0.35,80.0,7.5,M
823,Wipeout: The Game,wii,2009,misc,1.94,0.00,0.00,0.12,-1.0,-1.0,NaN
848,Rayman Raving Rabbids: TV Party,wii,2008,misc,0.72,1.08,0.00,0.23,73.0,7.7,E10+
...,...,...,...,...,...,...,...,...,...,...,...
16671,Fullmetal Alchemist: Prince of the Dawn,wii,2009,adventure,0.00,0.00,0.01,0.00,-1.0,-1.0,NaN
16753,Routes PE,ps2,2007,adventure,0.00,0.00,0.01,0.00,-1.0,-1.0,NaN
16799,Transformers: Prime,wii,2012,action,0.00,0.01,0.00,0.00,-1.0,-1.0,NaN
16912,Metal Gear Solid V: The Definitive Experience,xone,2016,action,0.01,0.00,0.00,0.00,-1.0,-1.0,M


Исходя из данных, явных дубликатов в данных нет

- В процессе подготовки данных вы могли что-либо удалять, например строки с пропусками или ошибками, дубликаты и прочее. В этом случае посчитайте количество удалённых строк в абсолютном и относительном значениях.

In [25]:
initial_rows = len(new_game)
print(f"Исходное количество строк: {initial_rows}")
#Удаление строк (например, строк с пропусками или дубликатов):
new_game_cleaned = new_game.dropna()
new_game_cleaned = new_game.drop_duplicates()
#Подсчёт количества строк после удаления:
cleaned_rows = len(new_game_cleaned)
print(f"Количество строк после очистки: {cleaned_rows}")
#Подсчёт абсолютного количества удалённых строк:
deleted_rows = initial_rows - cleaned_rows
print(f"Абсолютное количество удалённых строк: {deleted_rows}")
#Подсчёт относительного количества удалённых строк:
relative_deleted_rows = deleted_rows / initial_rows
print(f"Относительное количество удалённых строк: {relative_deleted_rows:.2%}")

Исходное количество строк: 16956
Количество строк после очистки: 16715
Абсолютное количество удалённых строк: 241
Относительное количество удалённых строк: 1.42%


В результате предобработки данных, были заменены типы данных в столбцах для более корректной работы и дальнейшего ознакомления с датасетом

Было удалено **241** строк, что составляет **1,42%** от исходного объёма данных. Это свидетельствует о том, что большая часть данных была сохранена, и потери информации минимальны.

Данные стали более чистыми и структурированными, что улучшит точность и надёжность последующих аналитических выводов.

---

## 3. Фильтрация данных

Коллеги хотят изучить историю продаж игр в начале XXI века, и их интересует период с 2000 по 2013 год включительно. Отберите данные по этому показателю. Сохраните новый срез данных в отдельном датафрейме, например `df_actual`.

In [26]:
#Отбор данных за период 2000-2013
df_actual = new_game.loc[new_game['year_of_release'].between(2000, 2013)].copy()
df_actual

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E
2,Mario Kart Wii,wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E
6,New Super Mario Bros.,ds,2006,platform,11.28,9.14,6.50,2.88,89.0,8.5,E
7,Wii Play,wii,2006,misc,13.96,9.18,2.93,2.84,58.0,6.6,E
...,...,...,...,...,...,...,...,...,...,...,...
16947,Men in Black II: Alien Escape,gc,2003,shooter,0.01,0.00,0.00,0.00,-1.0,-1.0,T
16949,Woody Woodpecker in Crazy Castle 5,gba,2002,platform,0.01,0.00,0.00,0.00,-1.0,-1.0,NaN
16950,SCORE International Baja 1000: The Official Game,ps2,2008,racing,0.00,0.00,0.00,0.00,-1.0,-1.0,NaN
16952,LMA Manager 2007,x360,2006,sports,0.00,0.01,0.00,0.00,-1.0,-1.0,NaN


Мы видим, этому периоду соответствуют 12980 строк, с этим датафреймом мы и будем дальше работать

---

## 4. Категоризация данных
    
Проведите категоризацию данных:
- Разделите все игры по оценкам пользователей и выделите такие категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала).

In [27]:
#Выделяем категории согласно оценок пользователей
def categorize_user_score(score):
    if 8 <= score <= 10:
        return 'высокая'
    elif 3 <= score < 8:
        return 'средняя'
    elif 0 <= score < 3:
        return 'низкая'
    else:
        'нет категории'
#Применим категорию к столбцу user_score и создадим новый столбец 'категория_оценки'
df_actual.loc[:, 'категория_оценки'] = df_actual['user_score'].apply(categorize_user_score)

#Выводим первые пять строк для проверки
df_actual[['name', 'user_score', 'категория_оценки']].head()

,name,user_score,категория_оценки
0,Wii Sports,8.0,высокая
2,Mario Kart Wii,8.3,высокая
3,Wii Sports Resort,8.0,высокая
6,New Super Mario Bros.,8.5,высокая
7,Wii Play,6.6,средняя


- Разделите все игры по оценкам критиков и выделите такие категории: высокая оценка (от 80 до 100 включительно), средняя оценка (от 30 до 80, не включая правую границу интервала) и низкая оценка (от 0 до 30, не включая правую границу интервала).

In [28]:
#Выделяем категории согласно оценок критиков
def categorize_critic_score(c_score):
    if 80 <= c_score <= 100:
        return 'высокая оценка'
    elif 30 < c_score < 80:
        return 'средняя оценка'
    elif 0 <= c_score <= 30:
        return 'низкая оценка'
#Применим категорию к столбцу critic_score и создадим новый столбец 'категория_оценки_критиков'
df_actual.loc[:,'категория_оценки_критиков'] = df_actual['critic_score'].apply(categorize_critic_score)

#Выводим первые пять строк для проверки
df_actual[['name', 'critic_score', 'категория_оценки_критиков']].head()

,name,critic_score,категория_оценки_критиков
0,Wii Sports,76.0,средняя оценка
2,Mario Kart Wii,82.0,высокая оценка
3,Wii Sports Resort,80.0,высокая оценка
6,New Super Mario Bros.,89.0,высокая оценка
7,Wii Play,58.0,средняя оценка


- После категоризации данных проверьте результат: сгруппируйте данные по выделенным категориям и посчитайте количество игр в каждой категории.

In [29]:
# Группируем данные по категории оценок пользователей и считаем количество игр в каждой категории
user_score_categories = df_actual.groupby('категория_оценки').size().reset_index(name='количество_игр')
user_score_categories

,категория_оценки,количество_игр
0,высокая,2307
1,низкая,117
2,средняя,4148


In [30]:
# Группируем данные по категории оценок критиков и считаем количество игр в каждой категории
critic_score_categories = df_actual.groupby('категория_оценки_критиков').size().reset_index(name='количество_игр')
critic_score_categories

,категория_оценки_критиков,количество_игр
0,высокая оценка,1712
1,низкая оценка,67
2,средняя оценка,5488


- Выделите топ-7 платформ по количеству игр, выпущенных за весь актуальный период.

In [31]:
# Группируем данные по платформе и считаем количество игр для каждой платформы
platform_counts = df_actual.groupby('platform').size().reset_index(name='количество_игр')
# Сортируем платформы по количеству игр в порядке убывания
platform_counts_sorted = platform_counts.sort_values(by='количество_игр', ascending=False)
# Выбираем топ-7 платформ
top_7_platforms = platform_counts_sorted.head(7)
top_7_platforms

,platform,количество_игр
9,ps2,2154
2,ds,2146
14,wii,1294
12,psp,1199
17,x360,1138
10,ps3,1107
4,gba,826


---

## 5. Итоговый вывод

В процессе анализа данных о продажах игр был проведён ряд операций по предобработке и категоризации данных. Удалены строки с пропусками, дубликатами или некорректными значениями, что улучшило качество данных. Были добавлены новые поля для категоризации оценок пользователей и критиков, что позволило сгруппировать игры по уровню оценок. Также был выполнен срез данных для анализа игр, выпущенных в период с 2000 по 2013 год, и определены топ-7 платформ по количеству выпущенных игр.

Проделанная работа:

1. Предобработка данных: удаление строк с пропусками и дубликатами, что обеспечило чистоту и структурированность данных.
2. Категоризация оценок: создание новых полей категория_оценки и категория_оценки_критиков для группировки игр по уровню оценок пользователей и критиков соответственно.
3. Срез данных: отбор данных о играх, выпущенных в период с 2000 по 2013 год, для более детального анализа.
4. Анализ платформ: определение топ-7 платформ по количеству выпущенных игр за указанный период.

Описание среза данных:

Срез данных включает информацию о играх, выпущенных в период с 2000 по 2013 год. Это позволяет сосредоточиться на анализе рынка видеоигр в начале XXI века и выявить тенденции и особенности этого периода.

Новые поля, добавленные в исходный датасет:

- категория_оценки: категоризация оценок пользователей по уровням «высокая», «средняя», «низкая».
- категория_оценки_критиков: категоризация оценок критиков по уровням «высокая оценка», «средняя оценка», «низкая оценка».